In [ ]:
import matplotlib.pyplot as plt
import json
import pandas as pd
import seaborn as sns
import re
from transformers import BertTokenizer, BertForSequenceClassification

json_file = 'gggg.json'
csv_file = 'data1.csv'

# Load JSON data
with open(json_file, 'r') as file:
    data = pd.read_json(file)

# Save as CSV
data.to_csv(csv_file, index=False)

# Step 2: Read CSV into DataFrame
df = pd.read_csv(csv_file)

In [ ]:
df

In [ ]:
### Data Visualization
plt.figure(figsize=(8, 6))
plt.scatter(df['stars'].value_counts().sort_index().index, df['stars'].value_counts().sort_index(), color='purple')
plt.title('Review Ratings')
plt.xlabel('Stars')
plt.ylabel('Count')
plt.show()

In [ ]:
# Plot the distribution of review lengths
plt.figure(figsize=(8, 6))
df['review_length'] = df['text'].apply(len)
df['review_length'].hist(bins=50, color='red')
plt.title('Review Lengths')
plt.xlabel('Review Length')
plt.ylabel('Amount')
plt.show()